### Exercises

### Question 1

There is a file named `transactions.csv` which is a list of purchases and sales.

Write code that loads this data and calculates the total of these purchases and sales.

Take two approaches - one using floats, and one using Decimal objects. Calculate the difference between the two results.

Also, time how long it takes to run your code using floats and using Decimals.

##### Solution 1

In [1]:
import csv
import decimal
from decimal import Decimal

In [2]:
file = 'transactions.csv'

In [32]:
# checking out what's in the file
with open(file) as f:
    reader = csv.reader(f)
    for _ in range(5):
        print(next(reader))

['timestamp', 'account', 'amount']
['2020-11-03T02:01:50', '6136306', '-11.022038']
['2020-06-19T07:32:00', '3369009', '-56.825416']
['2021-01-29T13:29:17', '4366765', '-87.430871']
['2020-03-31T09:27:11', '3298760', '16.161836']


In [33]:
# write function to calculate total of purchases and sales using floats and decimals
def ps_total(f_name, use_decimal=False):
    with open(f_name) as f:
        reader = csv.reader(f)
        next(reader) # skip header row
        return sum(Decimal(row[-1]) if use_decimal else float(row[-1]) for row in reader)

In [34]:
# lets use a decorator function to time the calculation of ps_total in float and decimals
def ps_decorator(fn):
    from time import perf_counter
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('elapsed time:', end - start)
        return result
    return inner

ps_total = ps_decorator(ps_total) # ps_total function decorated with time calculation features

In [35]:
ps_float = ps_total(file)
ps_float

elapsed time: 1.7656877330000498


116387.51306500046

In [36]:
ps_decimal = ps_total(file, use_decimal=True)
ps_decimal

elapsed time: 2.1473726210001587


Decimal('116387.513065')

In [37]:
# calculate difference in floats vs Decimal objects
format(abs(ps_float - float(ps_decimal)), '.27f')

'0.000000000451109372079372406'

calculating % time difference between using `float` and `Decimal`

In [38]:
from timeit import timeit

In [39]:
time_float = timeit('ps_total(file)', globals=globals(), number=5)
time_decimal = timeit('ps_total(file, use_decimal=True)', globals=globals(), number=5)

# calculate how much slower decimal is to float
time_diff = round((time_decimal - time_float) / time_float * 100, 1)
print(f'Decimal is slower than float by {time_diff}%')

elapsed time: 1.776984153000285
elapsed time: 1.6710079500003303
elapsed time: 1.6525091489997976
elapsed time: 1.7472758489998341
elapsed time: 1.63811002500006
elapsed time: 2.236509633999958
elapsed time: 2.1305055319999155
elapsed time: 2.1894125379999423
elapsed time: 2.1917400699999234
elapsed time: 2.199700372999814
Decimal is slower than float by 29.0%


#### Question 2

Using the same file (`transactions.csv`), we now want to calculate a fee on each transaction.

Irrespective of whether the transaction was a credit or a debit, we will calculate a `0.123%` transaction fee for the (absolute) values of each transaction.

**Each** fee calculation precision should be limited to `8` digits after the decimal point (so use `round(val, 8)`)

In addition, any rounding should always round ties away from `0` (`ROUND_HALF_UP`) - and not use Banker's rounding (`ROUND_HALF_EVEN`).

Only implement this solution using `Decimal` objects, as floats do not offer a rounding algorithm choice, and writing our own rounding algorithm can be overly complicated.

Also calculate the different in the fee totals when using `ROUND_HALF_UP` vs `ROUND_HALF_EVEN`

In [30]:
# write function to calculate transaction fee of purchases and sales using decimals
def ps_transaction_fee(f_name, t_fee=.123, rounding_even=False):
    t_fee = Decimal(f'{t_fee}')
    with open(f_name) as f:
        reader = csv.reader(f)
        next(reader) # skip header row
        amounts = [Decimal(row[-1]) for row in reader]
        _sum = lambda: sum(round(abs(amt * (t_fee/100)), 8) for amt in amounts)
        transaction_fees_EVEN = _sum()
        with decimal.localcontext() as ctx:
            ctx.rounding = decimal.ROUND_HALF_UP
            transaction_fees_UP = _sum()
    return transaction_fees_EVEN if rounding_even else transaction_fees_UP

In [31]:
ps_UP = ps_transaction_fee(file)
ps_EVEN = ps_transaction_fee(file, rounding_even=True)

print(ps_UP, ps_EVEN)
abs(ps_UP - ps_EVEN)

125501.66978197 125501.66977180


Decimal('0.00001017')